In [1]:
!pip install pyautogen tavily-python cohere ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 642.5/642.5 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.8 MB/s eta 0:00:00


In [2]:
import os
import json
import time
import autogen
from tavily import TavilyClient
import cohere
from typing import Dict, List, Optional, Any
from IPython.display import HTML, display
import ipywidgets as widgets

In [3]:
os.environ["COHERE_API_KEY"] = "yIX1KWTGw2HTvzTjEBuDE6Kib4YiSBnoD7RuCzGj"
os.environ["TAVILY_API_KEY"] = "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"

# --- Step 4: Initialize API clients ---
cohere_client = cohere.Client(os.environ["COHERE_API_KEY"])
tavily_client = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

# --- Step 5: Set up configuration for AutoGen ---
llm_config = {
    "config_list": [
        {
            "model": "command",  # Using Cohere's "command" model
            "api_key": os.environ["COHERE_API_KEY"],
            "provider": "cohere"
        }
    ],
    "temperature": 0.7
}

Autogen without human intervention

In [4]:
from tavily import TavilyClient
import cohere
from autogen import AssistantAgent, UserProxyAgent

# --- Tavily Web Search Setup ---
TAVILY_API_KEY = "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"
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

def search_web(query):
    results = tavily_client.search(query, max_results=5)
    research_text = "\n".join([result["content"] for result in results["results"]])

    print("\n--- Research Results ---\n", research_text)  # Print research results
    return research_text

# --- Cohere Content Generation & Translation ---
COHERE_API_KEY = "yIX1KWTGw2HTvzTjEBuDE6Kib4YiSBnoD7RuCzGj"
co = cohere.Client(COHERE_API_KEY)

def generate_article(research_text, style="formal"):
    response = co.generate(
        model="command",
        prompt=f"Write a well-structured article in {style} style based on the following research:\n{research_text}",
        max_tokens=1000
    )
    article = response.generations[0].text

    print("\n--- Generated Article ---\n", article)  # Print generated article
    return article

def translate_article(article_text, target_language="Spanish"):
    response = co.generate(
        model="command",
        prompt=f"Translate the following article to {target_language} while maintaining accuracy and tone:\n{article_text}",
        max_tokens=1000
    )
    translated_article = response.generations[0].text

    print("\n--- Translated Article ---\n", translated_article)  # Print translated article
    return translated_article

# --- AutoGen Implementation ---
def autogen_pipeline(topic, style="formal", target_language="Spanish"):
    user = UserProxyAgent("User", max_consecutive_auto_reply=5)

    assistant = AssistantAgent(
        "AI_Assistant",
        llm_config={
            "model": "command",
            "api_key": COHERE_API_KEY,
            "provider": "cohere"
        }
    )

    research_text = search_web(topic)
    article = generate_article(research_text, style)
    translated_article = translate_article(article, target_language)

    return research_text, article, translated_article  # Return all three outputs

# Example usage
topic = "Latest AI Trends"
research_text, article, translated_article = autogen_pipeline(topic)


[autogen.oai.client: 03-12 18:03:19] {307} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



--- Research Results ---
 Top 15 AI Trends for 2025: Expert Predictions You Need to Know - Tech Startups Deepseek, a free open-source AI model developed by a Chinese tech startup, exemplifies a growing trend in open-source AI, where accessible tools are pushing the boundaries of performance and affordability. This competition is as much about technological dominance as it is about shaping global AI policies and addressing challenges like cybersecurity, data privacy, and ethics. Workplace Productivity: AI-powered tools like scheduling assistants, email managers, and task prioritization systems will free professionals from mundane tasks. By 2025, self-learning systems will likely play a significant role in automating the development of more sophisticated AI models, reducing the need for human intervention in fine-tuning and training. AI in Cybersecurity
Our four hot trends to watch out for in 2024 included what we called customized chatbots—interactive helper apps powered by multimodal 

Autogen with Human Intervention

In [ ]:
import os
from tavily import TavilyClient
import cohere
from autogen import AssistantAgent, UserProxyAgent

# --- Set Cohere API Key as an Environment Variable ---
os.environ["COHERE_API_KEY"] = "yIX1KWTGw2HTvzTjEBuDE6Kib4YiSBnoD7RuCzGj"

# --- Tavily Web Search Setup ---
tavily_client = TavilyClient(api_key="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")

def search_web(query):
    results = tavily_client.search(query, max_results=5)
    research_text = "\n".join([result["content"] for result in results["results"]])
    print("\n--- Research Results ---\n", research_text)
    input("Press Enter to proceed or edit the research results: ")
    return research_text

# --- Cohere Content Generation & Translation ---
co = cohere.Client(os.environ["COHERE_API_KEY"])

def generate_article(research_text, style="formal"):
    response = co.generate(
        model="command",
        prompt=f"Write a well-structured article in {style} style based on the following research:\n{research_text}",
        max_tokens=1000
    )
    article = response.generations[0].text
    print("\n--- Generated Article ---\n", article)
    user_input = input("Press Enter to proceed or modify the article: ")
    if user_input.strip():
        article = user_input  # Allow manual editing
    return article

def translate_article(article_text, target_language="Spanish"):
    response = co.generate(
        model="command",
        prompt=f"Translate the following article to {target_language} while maintaining accuracy and tone:\n{article_text}",
        max_tokens=1000
    )
    translated_article = response.generations[0].text
    print("\n--- Translated Article ---\n", translated_article)
    user_input = input("Press Enter to proceed or modify the translation: ")
    if user_input.strip():
        translated_article = user_input  # Allow manual editing
    return translated_article

# --- AutoGen Implementation ---
def autogen_pipeline(topic, style="formal", target_language="Spanish"):
    user = UserProxyAgent("User", max_consecutive_auto_reply=5)

    assistant = AssistantAgent(
        "AI_Assistant",
        llm_config={
            "model": "command",
            "api_key": os.environ["COHERE_API_KEY"],  # Ensure Cohere API key is used
            "provider": "cohere"
        }
    )

    research_text = search_web(topic)
    article = generate_article(research_text, style)
    translated_article = translate_article(article, target_language)
    return translated_article

# Example usage
topic = "Latest AI Trends"
translated_article_autogen = autogen_pipeline(topic)


[autogen.oai.client: 03-12 18:04:41] {307} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.



--- Research Results ---
 Top 15 AI Trends for 2025: Expert Predictions You Need to Know - Tech Startups Deepseek, a free open-source AI model developed by a Chinese tech startup, exemplifies a growing trend in open-source AI, where accessible tools are pushing the boundaries of performance and affordability. This competition is as much about technological dominance as it is about shaping global AI policies and addressing challenges like cybersecurity, data privacy, and ethics. Workplace Productivity: AI-powered tools like scheduling assistants, email managers, and task prioritization systems will free professionals from mundane tasks. By 2025, self-learning systems will likely play a significant role in automating the development of more sophisticated AI models, reducing the need for human intervention in fine-tuning and training. AI in Cybersecurity
Our four hot trends to watch out for in 2024 included what we called customized chatbots—interactive helper apps powered by multimodal 